<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.4.4


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 6s 203ms/step - loss: 0.6791 - accuracy: 0.5526 - val_loss: 0.6760 - val_accuracy: 0.4439
Epoch 2/10
26/26 [==============================] - 5s 179ms/step - loss: 0.6228 - accuracy: 0.6691 - val_loss: 0.5821 - val_accuracy: 0.8049
Epoch 3/10
26/26 [==============================] - 5s 179ms/step - loss: 0.5775 - accuracy: 0.7627 - val_loss: 0.5093 - val_accuracy: 0.7854
Epoch 4/10
26/26 [==============================] - 5s 179ms/step - loss: 0.4989 - accuracy: 0.7843 - val_loss: 0.5265 - val_accuracy: 0.7220
Epoch 5/10
26/26 [==============================] - 5s 180ms/step - loss: 0.5403 - accuracy: 0.7383 - val_loss: 0.4338 - val_accuracy: 0.8293
Epoch 6/10
26/26 [==============================] - 5s 179ms/step - loss: 0.4106 - accuracy: 0.8265 - val_loss: 0.3339 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 5s 180ms/step - loss: 0.3541 - accuracy: 0.8528 - val_loss: 0.3320 - val_accuracy: 0.8878
Epoch 

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 5s - loss: 0.6770 - accuracy: 0.5499 - val_loss: 0.6585 - val_accuracy: 0.7610

Epoch 00001: saving model to weights.01-0.66.h5
Epoch 2/5
26/26 - 5s - loss: 0.6220 - accuracy: 0.7226 - val_loss: 0.6189 - val_accuracy: 0.7073

Epoch 00002: saving model to weights.02-0.62.h5
Epoch 3/5
26/26 - 5s - loss: 0.5600 - accuracy: 0.7482 - val_loss: 0.6137 - val_accuracy: 0.6244

Epoch 00003: saving model to weights.03-0.61.h5
Epoch 4/5
26/26 - 5s - loss: 0.5103 - accuracy: 0.7810 - val_loss: 0.5777 - val_accuracy: 0.6927

Epoch 00004: saving model to weights.04-0.58.h5
Epoch 5/5
26/26 - 5s - loss: 0.4558 - accuracy: 0.7908 - val_loss: 0.4358 - val_accuracy: 0.8293

Epoch 00005: saving model to weights.05-0.44.h5


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 5s - loss: 0.6776 - accuracy: 0.5876 - val_loss: 0.6600 - val_accuracy: 0.5463

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2
26/26 - 5s - loss: 0.6712 - accuracy: 0.5949 - val_loss: 0.6484 - val_accuracy: 0.6732

Epoch 00001: saving model to model.h5
Epoch 2/2
26/26 - 5s - loss: 0.6305 - accuracy: 0.7056 - val_loss: 0.6104 - val_accuracy: 0.8195

Epoch 00002: saving model to model.h5


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 5s - loss: 0.6766 - accuracy: 0.5450 - val_loss: 0.6648 - val_accuracy: 0.5366
Epoch 2/50
26/26 - 5s - loss: 0.6332 - accuracy: 0.6582 - val_loss: 0.6053 - val_accuracy: 0.6537
Epoch 3/50
26/26 - 5s - loss: 0.5791 - accuracy: 0.7762 - val_loss: 0.5456 - val_accuracy: 0.7171
Epoch 4/50
26/26 - 5s - loss: 0.5154 - accuracy: 0.7981 - val_loss: 0.4810 - val_accuracy: 0.7659
Epoch 5/50
26/26 - 5s - loss: 0.4454 - accuracy: 0.8236 - val_loss: 0.4030 - val_accuracy: 0.8098
Epoch 6/50
26/26 - 5s - loss: 0.3560 - accuracy: 0.8832 - val_loss: 0.3033 - val_accuracy: 0.9073
Epoch 7/50
26/26 - 5s - loss: 0.2757 - accuracy: 0.9148 - val_loss: 0.2262 - val_accuracy: 0.9415
Epoch 8/50
26/26 - 5s - loss: 0.2197 - accuracy: 0.9404 - val_loss: 0.1616 - val_accuracy: 0.9756
Epoch 9/50
26/26 - 5s - loss: 0.1636 - accuracy: 0.9635 - val_loss: 0.1481 - val_accuracy: 0.9561
Epoch 10/50
26/26 - 5s - loss: 0.1440 - accuracy: 0.9611 - val_loss: 0.1311 - val_accuracy: 0.9610
Epoch 11/50
26/26 -

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 5s 177ms/step - loss: 0.6655 - accuracy: 0.5962 - val_loss: 0.7176 - val_accuracy: 0.4585
Epoch 2/5
26/26 [==============================] - 5s 177ms/step - loss: 0.5904 - accuracy: 0.6762 - val_loss: 0.6218 - val_accuracy: 0.6049
Epoch 3/5
26/26 [==============================] - 5s 179ms/step - loss: 0.5551 - accuracy: 0.7318 - val_loss: 0.4892 - val_accuracy: 0.8293
Epoch 4/5
26/26 [==============================] - 5s 173ms/step - loss: 0.4867 - accuracy: 0.7700 - val_loss: 0.4554 - val_accuracy: 0.8390
Epoch 5/5
26/26 [==============================] - 5s 177ms/step - loss: 0.4064 - accuracy: 0.8337 - val_loss: 0.3446 - val_accuracy: 0.8732


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.616788,0.656803,0.458537,0.717647
1,1,0.714112,0.578404,0.604878,0.621806
2,2,0.777372,0.523415,0.829268,0.489194
3,3,0.787105,0.467757,0.839024,0.455404
4,4,0.827251,0.408216,0.873171,0.344640


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 5s 184ms/step - loss: 0.6811 - accuracy: 0.5141 - val_loss: 0.6458 - val_accuracy: 0.7561
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 5s 172ms/step - loss: 0.6389 - accuracy: 0.7032 - val_loss: 0.6309 - val_accuracy: 0.7073
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 5s 180ms/step - loss: 0.6176 - accuracy: 0.7187 - val_loss: 0.6342 - val_accuracy: 0.6293
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 5s 175ms/step - loss: 0.6049 - accuracy: 0.7215 - val_loss: 0.6197 - val_accuracy: 0.6976
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 5s 176ms/step - loss: 0.6048 -

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 5s 184ms/step - loss: 0.6785 - accuracy: 0.5543 - val_loss: 0.6448 - val_accuracy: 0.6000
Epoch 2/50
26/26 [==============================] - 5s 175ms/step - loss: 0.6353 - accuracy: 0.6198 - val_loss: 0.6413 - val_accuracy: 0.5512
Epoch 3/50
26/26 [==============================] - 5s 175ms/step - loss: 0.5884 - accuracy: 0.6945 - val_loss: 0.5359 - val_accuracy: 0.8098
Epoch 4/50
26/26 [==============================] - 5s 180ms/step - loss: 0.5430 - accuracy: 0.7955 - val_loss: 0.5239 - val_accuracy: 0.7415
Epoch 5/50
26/26 [==============================] - 5s 172ms/step - loss: 0.4639 - accuracy: 0.7983 - val_loss: 0.4198 - val_accuracy: 0.8488
Epoch 6/50
26/26 [==============================] - 5s 179ms/step - loss: 0.3984 - accuracy: 0.8540 - val_loss: 0.3388 - val_accuracy: 0.8927
Epoch 7/50
26/26 [==============================] - 5s 175ms/step - loss: 0.3701 - accuracy: 0.8472 - val_loss: 0.3131 - val_accuracy: 0.8976
Epoch 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 11722), started 0:04:02 ago. (Use '!kill 11722' to kill it.)